![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_in_Spark_NLP_LongformerForSequenceClassification.ipynb)

## Import LongformerForSequenceClassification models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 4.0.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import Longformer models trained/fine-tuned for sequence classification via `LongformerForSequenceClassification` or `TFLongformerForSequenceClassification`. These models are usually under `Text Clasification` category and have `longformer` in their labels
- Reference: [TFLongformerForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/longformer#transformers.TFLongformerForSequenceClassification)
- Some [example models](https://huggingface.co/models?filter=longformer&pipeline_tag=text-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.7.1` version and Transformers on `4.19.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- DeBERTa v2&v3 use SentencePiece, so we will have to install that as well


In [1]:
!pip install -q transformers==4.39.3 tensorflow==2.11.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 32.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorflow-datasets 4.9.4 requires protobuf>=3.20, but you have

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [jpwahle/longformer-base-plagiarism-detection](https://huggingface.co/valhalla/hewonty/jpwahle/longformer-base-plagiarism-detection) model from HuggingFace as an example
- In addition to `TFLongformerForSequenceClassification` we also need to save the `LongformerTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [2]:
from transformers import TFLongformerForSequenceClassification, LongformerTokenizer
import tensorflow as tf

MODEL_NAME = 'jpwahle/longformer-base-plagiarism-detection'

tokenizer = LongformerTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFLongformerForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  model = TFLongformerForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical value

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {MODEL_NAME}

total 581060
-rw-r--r-- 1 root root      1023 Apr 12 12:24 config.json
drwxr-xr-x 3 root root      4096 Apr 12 12:23 saved_model
-rw-r--r-- 1 root root 594992384 Apr 12 12:24 tf_model.h5


In [4]:
!ls -l {MODEL_NAME}/saved_model/1

total 74792
drwxr-xr-x 2 root root     4096 Apr 12 12:23 assets
-rw-r--r-- 1 root root       53 Apr 12 12:24 fingerprint.pb
-rw-r--r-- 1 root root   201835 Apr 12 12:24 keras_metadata.pb
-rw-r--r-- 1 root root 76368775 Apr 12 12:24 saved_model.pb
drwxr-xr-x 2 root root     4096 Apr 12 12:23 variables


In [5]:
!ls -l {MODEL_NAME}_tokenizer

total 1432
-rw-r--r-- 1 root root 456318 Apr 12 12:18 merges.txt
-rw-r--r-- 1 root root    958 Apr 12 12:18 special_tokens_map.json
-rw-r--r-- 1 root root   1195 Apr 12 12:18 tokenizer_config.json
-rw-r--r-- 1 root root 999355 Apr 12 12:18 vocab.json


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `vocab.json` and `merges.txt` files from the tokenizer
- All we need is to first convert vocab.json to `vocab.txt` and copy both `vocab.txt` and `merges.txt` into saved_model/1/assets which Spark NLP will look for

In [6]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's save the vocab as txt file
with open('{}_tokenizer/vocab.txt'.format(MODEL_NAME), 'w') as f:
    for item in tokenizer.get_vocab().keys():
        f.write("%s\n" % item)


# get label strings
labels = [model.config.id2label[l] for l, v in model.config.id2label.items()]
print(labels)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

# let's copy both vocab.txt and merges.txt files to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/merges.txt {asset_path}
!cp {MODEL_NAME}_tokenizer/vocab.txt {asset_path}

['ORIGINAL', 'PLAGIARISM']


Voila! We have our `vocab.txt`, `merges.txt` and `labels.txt` inside assets directory

In [7]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 852
-rw-r--r-- 1 root root     19 Apr 12 12:24 labels.txt
-rw-r--r-- 1 root root 456318 Apr 12 12:24 merges.txt
-rw-r--r-- 1 root root 407065 Apr 12 12:24 vocab.txt


## Import and Save LongformerForSequenceClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [8]:
print('Restrt here')
while True:
  pass

Restrt here


KeyboardInterrupt: 

In [1]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-04-12 12:25:59--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-04-12 12:26:00--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 an

Let's start Spark with Spark NLP included via our simple `start()` function

In [2]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `LongformerForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `LongformerForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [3]:
from sparknlp.annotator import *
from sparknlp.base import *

MODEL_NAME = 'jpwahle/longformer-base-plagiarism-detection'

sequenceClassifier = LongformerForSequenceClassification.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
.setInputCols(['token', 'document']) \
.setOutputCol('class') \
.setCaseSensitive(True) \
.setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [4]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [5]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your LongformerForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [6]:
! ls -l {MODEL_NAME}_spark_nlp

total 655228
drwxr-xr-x 6 root root      4096 Apr 12 12:29 fields
-rw-r--r-- 1 root root 670939545 Apr 12 12:30 longformer_classification_tensorflow
drwxr-xr-x 2 root root      4096 Apr 12 12:29 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny LongformerForSequenceClassification model in Spark NLP 🚀 pipeline!

In [7]:
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier
])

# couple of simple examples
example = spark.createDataFrame([["My name is Wolfgang and I live in Berlin"]]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "class.result").show(truncate=False)

+----------------------------------------+----------+
|text                                    |result    |
+----------------------------------------+----------+
|My name is Wolfgang and I live in Berlin|[ORIGINAL]|
+----------------------------------------+----------+



That's it! You can now go wild and use hundreds of `LongformerForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
